# What is a _for_ loop?

## Classical _for_

In [2]:
for i in range(5):
    # Body of for loop
    print(i)

0
1
2
3
4


## Under the hood

How can we customize its behavior? To this end, we need to know what is under the hood of this syntax. This little script is roughly equivalent to the _for_ loop.

In [11]:
generator = range(5) # Instanciating the 'range' class returns a 'generator'
iterator = iter(generator) # A generator must return an iterator when we call 'iter' on it.
while True:
    try:
        i = next(iterator) # Calling 'next' on 'iterator' returns the next value in line.
    except StopIteration: # Once the last value has been reached, a StopIteration exception is raised
        break
    
    # Body of for loop
    print(i)

0
1
2
3
4


# Generator/Iterator classes

## What is a generator?
A generator is a class that defines the '\_\_iter__' method

In [20]:
class Generator:
    """
    Defines the __iter__ method.
    """
    def __iter__(self):
        print('Creating an iterator')
        return Iterator()

## What is an iterator?
An iterator is a class that defines the '\_\_next__' method.

In [23]:
class Iterator:
    """
    Defines the __next__ method.
    """
    def __next__(self):
        """
        Should raise StopIteration when iteration is finished.
        """
        print('Raising StopIteration')
        raise StopIteration

This is the minimum necessary to customize the behavior of the for loop:

In [24]:
for i in Generator():
    pass

Creating an iterator
Raising StopIteration


## Generator-Iterator

Generators and iterators are intertwined notions, since they can be both at the same time 

In [25]:
class GeneratorIterator:
    def __iter__(self):
        # Initialize the iterator
        print('The iterator is itself')
        return self
    def __next__(self):
        print('Raising StopIteration')
        raise StopIteration

In [26]:
for i in GeneratorIterator():
    pass

The iterator is itself
Raising StopIteration


# Examples with the '\_\_next__' implementation

Let us show how this works by simulating common `Python` generators.

## The _range_ generator

In [47]:
class Range:
    """
    Emulation of 'range' generator.
    """
    def __init__(self, start, stop=None, stride=1):
        if stop is None:
            stop = start
            start = 0
        self.start = start
        self.stop = stop
        self.stride = stride

    def __iter__(self):
        self.current = self.start - self.stride # minus self.stride to compensate the first increment in __next__.
        return self
        
    def __next__(self):
        self.current += self.stride # Increments self.current
        if self.current >= self.stop:
            raise StopIteration
        return self.current
    
    def __len__(self):
        return (self.stop - self.start)//self.stride

In [48]:
for i in Range(3,10,2):
    print(i)

3
5
7
9


## The _zip_ generator

In [63]:
class Zip:
    """
    Emulation of the 'zip' generator.
    """
    def __init__(self, *generators):
        self.generators = generators

    def __iter__(self):
        self.iterators = [iter(gen) for gen in self.generators] # Initializing every sub-iterators
        return self

    def __next__(self):
        return tuple([next(it) for it in self.iterators]) # Calling next on all sub-iterators

In [64]:
for i, j in Zip(Range(3,10,2), Range(6)):
    print(i, j)

3 0
5 1
7 2
9 3


Note how the _Zip_ generator stops with the shortest generator. It stops at the first _StopIteration_ raised.

## The _enumerate_ generator

In [69]:
class Enumerate:
    """
    Emulation of the 'enumerate' generator.
    """
    def __init__(self, generator):
        self.generator = generator

    def __iter__(self):
        self.i = -1
        self.iterator = iter(self.generator)
        return self

    def __next__(self):
        self.i += 1
        return (self.i, next(self.iterator))

In [72]:
for i, (a, b) in Enumerate(Zip(Range(3,10,2), Range(2,6))):
    print(i, a, b)

0 3 2
1 5 3
2 7 4
3 9 5


Note how we can unpack the tuple directly in the declaration of the _for_ loop.

# The _yield_ statement

There is an easier and simpler way to define (short) generator in `Python` using the keyword _yield_. It uses the classical syntax of a function definition.

In [74]:
def range_generator(stop):
    i = 0
    while i < stop:
        yield i
        i += 1
    # No return statement will implicitely return None

In [75]:
for i in range_generator(5):
    print(i)

0
1
2
3
4


This generator will execute the function until it encounters a yield. It will then hang there until the next() call, where the rest of the function will be executed. When it runs into a _return_, the following next() call will raise StopIteration. Note that if a function does not have a return statement, an implicit None will be returned at the end.

In [76]:
gen = range_generator(5)
it = iter(gen)
while True:
    next(it) # Will raise a StopIteration exception

StopIteration: 

## Examples revisited with the _yield_ statement

The _yield_ statement can simplify greatly some generator and are recommended when they can be used. The \_\_next__ method should be used only when the _yield_ generator gets complicated and separating the two clarifies the code, or in case there are conflict with older code, since in `Python 2.x`, _yield_ must be imported from \_\_future__.

### The _range_ generator

In [81]:
class RangeYield:
    """
    Emulation of 'range' generator using the 'yield' statement.
    """
    def __init__(self, start, stop=None, stride=1):
        if stop is None:
            stop = start
            start = 0
        self.start = start
        self.stop = stop
        self.stride = stride

    def __iter__(self):
        current = self.start
        while current < self.stop:
            yield current
            current += self.stride

    def __len__(self):
        return (self.stop - self.start)//self.stride

In [82]:
for i in RangeYield(3,10,2):
    print(i)

3
5
7
9


### The _zip_ generator

In [84]:
class ZipYield:
    """
    Emulation of the 'zip' generator using the 'yield' statement.
    """
    def __init__(self, *generators):
        self.generators = generators

    def __iter__(self):
        iterators = [iter(gen) for gen in self.generators] # Initializing every sub-iterators
        while True:
            return tuple([next(iterator) for iterator in iterators])

In [85]:
for i, j in Zip(Range(3,10,2), Range(6)):
    print(i, j)

3 0
5 1
7 2
9 3


### The _enumerate_ generator

In [86]:
class Enumerate:
    """
    Emulation of the 'enumerate' generator using the 'yield' statement.
    """
    def __init__(self, generator):
        self.generator = generator

    def __iter__(self):
        i = 0
        for val in self.generator:
            yield (i, val)
            i += 1

In [87]:
for i, (a, b) in Enumerate(Zip(Range(3,10,2), Range(2,6))):
    print(i, a, b)

0 3 2
1 5 3
2 7 4
3 9 5


## The _yield from_ statement

`Python 3.3` introduces the _yield from_ statement to delegate generators to other generators. They simply a little bit the syntax, but they are mainly useful for more advanced use of generators (coroutines with the methods iterator.send() and iterator.throw()).

The following example:

In [96]:
def pyramid_yield_from(max_val):
    yield from range(max_val)
    yield from range(max_val, -1, -1)
    
for i in pyramid_yield_from(3):
    print(i)

0
1
2
3
2
1
0


is roughly equivalent to

In [97]:
def pyramid_yield(max_val):
    for i in range(max_val):
        yield i
    for i in range(max_val, -1, -1):
        yield i
    
for i in pyramid_yield(3):
    print(i)

0
1
2
3
2
1
0
